In [24]:
def dilution_eq(V_1, M_1, V_2, M_2):
    if V_1 == None:
        return (V_2*M_2) / M_1
    elif M_2 == None:
        return round((V_1*M_1) / V_2, 2)

def adjust_take_vol(stock_conc, reac_vol, desired_conc):
    tolerance = 1e-2
    vol_take  = round(dilution_eq(None, stock_conc, reac_vol, desired_conc), 2)

    while True:
        total_vol = vol_take + reac_vol
        act_vol_take = dilution_eq(None, stock_conc, total_vol, desired_conc)
        if ( abs(act_vol_take - vol_take) <= tolerance ):
            break
        vol_take = round(act_vol_take, 2)
    return vol_take

def enzyme_vol(n_per_tube, amount_each_DNA):
    # 1 uL of enzyme per 100 pmol of DNA <=> x*0.01 uL -> x pmol
    return n_per_tube*0.01*amount_each_DNA

print('Number of DNAs per tube?')
n_per_tube = float(input())
print('Amount of each DNA (in pmol)?')
amount_each_DNA = float(input())

#############################################################
# STEP 1: PHOSPHORYLATION OF TEMPLATE STRAND

# 10 uL -> 100 pmol <=> x*0.1 uL -> x pmol
vol_each_DNA = amount_each_DNA*0.1
total_DNA_vol = n_per_tube*vol_each_DNA

T4_PNK_vol = enzyme_vol(n_per_tube, amount_each_DNA)
tot_reac_vol_1 = total_DNA_vol + T4_PNK_vol
ligase_buffer = adjust_take_vol(10, tot_reac_vol_1, 1)
tot_reac_vol_1 += ligase_buffer
each_DNA_conc_1 = round((100 / tot_reac_vol_1), 2) # assume 10 uM in 10 uL

print("\nStep 1: Phosphorylation of template strand")
print("\tVolume of DNA per tube: " + str(total_DNA_vol) + " uL")
print("\tVolume of 10X T4 DNA ligase buffer per tube: " + str(ligase_buffer) + " uL")
print("\tVolume of T4 PNK per tube: " + str(T4_PNK_vol) + " uL")
print("\tConcentration of each DNA: " + str(each_DNA_conc_1) + " uM")
print("\tTotal reaction volume per tube: " + str(tot_reac_vol_1) + " uL")

#############################################################
# STEP 2: SPLINT ANNEALING
print('\nSplint stock concentration (in uM)?')
splint_stock_conc = float(input())

# want 5 fold excess of splints
amount_of_splint_per_DNA = 5*amount_each_DNA
splint_vol_per_DNA = amount_of_splint_per_DNA / splint_stock_conc
splint_vol = n_per_tube*splint_vol_per_DNA

# make ligase buffer concentration 1X again
ligase_buffer = adjust_take_vol(10, splint_vol, 1)

tot_reac_vol_2 = splint_vol + ligase_buffer + tot_reac_vol_1
each_DNA_conc_2 = dilution_eq(tot_reac_vol_1, each_DNA_conc_1, tot_reac_vol_2, None)
splint_conc = dilution_eq(splint_vol_per_DNA, splint_stock_conc, tot_reac_vol_2, None)
aliquot = round((2.5 / each_DNA_conc_2), 2)
tot_reac_vol_2 -= aliquot

print("\nStep 2: Splint annealing")
print("\tVolume of splints per tube: " + str(splint_vol) + " uL")
print("\tVolume of 10X T4 DNA ligase buffer per tube: " + str(ligase_buffer) + " uL")
print("\tConcentration of each DNA: " + str(each_DNA_conc_2) + " uM")
print("\tConcentration of splints per " + str(amount_each_DNA) + " pmol of each DNA: " + str(splint_conc) + " uM")
print("\tTake " + str(aliquot) + " uL for 2.5 pmol aliquot")
print("\tTotal reaction volume per tube: " + str(tot_reac_vol_2) + " uL")

#############################################################
# STEP 3: LIGATION OF ssMC
T4_DNA_lig_vol = enzyme_vol(n_per_tube, amount_each_DNA)

# make ligase buffer concentration 1X again
ligase_buffer = adjust_take_vol(10, T4_DNA_lig_vol, 1)

tot_reac_vol_3 = T4_DNA_lig_vol + ligase_buffer + tot_reac_vol_2
each_DNA_conc_3 = dilution_eq(tot_reac_vol_2, each_DNA_conc_2, tot_reac_vol_3, None)
aliquot = round((2.5 / each_DNA_conc_3), 2)
tot_reac_vol_3 -= aliquot

print("\nStep 3: Ligation of single stranded minicircles")
print("\tVolume of 10X T4 DNA ligase buffer per tube: " + str(ligase_buffer) + " uL")
print("\tVolume of T4 DNA ligase per tube: " + str(T4_DNA_lig_vol) + " uL")
print("\tConcentration of each DNA: " + str(each_DNA_conc_3) + " uM")
print("\tTake " + str(aliquot) + " uL for 2.5 pmol aliquot")
print("\tTotal reaction volume per tube: " + str(tot_reac_vol_3) + " uL")

#############################################################
# STEP 4: PHOSPHORYLATION OF COMPLEMENTARY STRANDS

# want 2 fold excess of complementary for every 100 pmol of each DNA
amount_of_comp_per_DNA = 2*amount_each_DNA
comp_vol_per_DNA = amount_of_comp_per_DNA / 10 # assume 10 uM in 10 uL
comp_vol = n_per_tube*comp_vol_per_DNA

# make ligase buffer concentration 1X again
ligase_buffer = adjust_take_vol(10, comp_vol, 1)

T4_PNK_vol = enzyme_vol(n_per_tube, amount_of_comp_per_DNA)
tot_reac_vol_4 = ligase_buffer + T4_PNK_vol + comp_vol
comp_conc_per_DNA = dilution_eq(comp_vol_per_DNA, 10, tot_reac_vol_4, None)

print("\nStep 4: Phosphorylation of complementary strand")
print("\tVolume of complementary per tube: " + str(comp_vol) + " uL")
print("\tVolume of 10X T4 DNA ligase buffer per tube: " + str(ligase_buffer) + " uL")
print("\tVolume of T4 PNK per tube: " + str(T4_PNK_vol) + " uL")
print("\tConcentration of complementary strands per DNA: " + str(comp_conc_per_DNA) + " uM")
print("\tTotal reaction volume per tube: " + str(tot_reac_vol_4) + " uL")

Number of DNAs per tube?
5
Amount of each DNA (in pmol)?
100

Step 1: Phosphorylation of template strand
	Volume of DNA per tube: 50.0 uL
	Volume of 10X T4 DNA ligase buffer per tube: 6.11 uL
	Volume of T4 PNK per tube: 5.0 uL
	Concentration of each DNA: 1.64 uM
	Total reaction volume per tube: 61.11 uL

Splint stock concentration (in uM)?
100

Step 2: Splint annealing
	Volume of splints per tube: 25.0 uL
	Volume of 10X T4 DNA ligase buffer per tube: 2.77 uL
	Concentration of each DNA: 1.13 uM
	Concentration of splints per 100.0 pmol of each DNA: 5.63 uM
	Take 2.21 uL for 2.5 pmol aliquot
	Total reaction volume per tube: 86.67 uL

Step 3: Ligation of single stranded minicircles
	Volume of 10X T4 DNA ligase buffer per tube: 0.55 uL
	Volume of T4 DNA ligase per tube: 5.0 uL
	Concentration of each DNA: 1.06 uM
	Take 2.36 uL for 2.5 pmol aliquot
	Total reaction volume per tube: 89.86 uL

Step 4: Phosphorylation of complementary strand
	Volume of complementary per tube: 100.0 uL
	Volume of 